In [37]:
from pathlib import Path
import pandas as pd
import csv
import os
from dotenv import load_dotenv
#from googleapiclient.discovery import build

In [38]:
# Construct the file path using Path from pathlib
watchLaterVideo = Path("Takeout") / "YouTube and YouTube Music" / "playlists" / "Watch later videos.csv"

# Check if the file exists
if not watchLaterVideo.exists():
    print(f"File not found: {watchLaterVideo}")
    exit(1)
else:
    print(f"File found: {watchLaterVideo}")
# Check if the file is a CSV file
if watchLaterVideo.suffix != '.csv':
    print(f"File is not a CSV: {watchLaterVideo}")
    exit(1)
else:
    print(f"File is a CSV: {watchLaterVideo}")


File found: Takeout/YouTube and YouTube Music/playlists/Watch later videos.csv
File is a CSV: Takeout/YouTube and YouTube Music/playlists/Watch later videos.csv


In [39]:
from googleapiclient.discovery import build

load_dotenv()
GOOGLE_KEY = os.getenv("googleKey")
if not GOOGLE_KEY:
    print("Google API key not found in environment variables.")
    exit(1)
youtube = build("youtube", "v3", developerKey=GOOGLE_KEY)

In [40]:
def get_channel_info(video_id):
    try:
        # Get video details
        video_response = youtube.videos().list(
            part="snippet",
            id=video_id
        ).execute()

        if video_response['items']:
            return {
                # retruning the video title
                "title": video_response["items"][0]["snippet"]["title"],
                "channelId": video_response["items"][0]["snippet"]["channelId"],
                "channelTitle": video_response["items"][0]["snippet"]["channelTitle"]
            }
        
    except Exception as e:
        print(f"API Error: {str(e)}")
    return None

In [32]:
from datetime import datetime
from zoneinfo import ZoneInfo

t = '2025-04-18T08:25:32+00:00'

# Parse the string into a datetime object
dt = datetime.fromisoformat(t)

# Convert to the desired timezone
dt = dt.astimezone(ZoneInfo("Asia/Hong_Kong"))
print(f"Current date and time: {dt.strftime('%Y-%m-%d %H:%M:%S')}")

Current date and time: 2025-04-18 16:25:32


In [41]:
from datetime import datetime
from zoneinfo import ZoneInfo
# Function to convert timestamp to UTC
def convert_to_utc(timestamp):
    # Assuming the timestamp is in ISO 8601 format
    dt = datetime.fromisoformat(timestamp)
    # Convert to UTC
    dt_utc = dt.astimezone(ZoneInfo("UTC"))
    return dt_utc

# Read the CSV file and extract Video IDs
ids = []
urls = []
created_dates = []
with open(watchLaterVideo, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Check if the row has the required columns
        if 'Video ID' in row and 'Playlist video creation timestamp' in row:
            # Append the values to the lists
                ids.append(row['Video ID'])
                urls.append('https://www.youtube.com/watch?v' + row['Video ID'])
                # convert the timestamp to a more readable format
                timestamp = row['Playlist video creation timestamp']
                dt = datetime.fromisoformat(timestamp)
                dt = dt.astimezone(ZoneInfo("Asia/Hong_Kong"))
                dt = dt.strftime('%Y-%m-%d %H:%M:%S')
                created_dates.append(dt)
        else:
            print("Required columns not found in the CSV file.")
            exit(1)
# Print the first 5 entries of each list
print(ids[:5])
print(urls[:5])
print(created_dates[:5])

['vgmqUhvQlww', 'YfhPJ8GxdgI', '2aGe20OuUAU', 'JJXEhmcQxwg', 'vg8yizC4W8E']
['https://www.youtube.com/watch?vvgmqUhvQlww', 'https://www.youtube.com/watch?vYfhPJ8GxdgI', 'https://www.youtube.com/watch?v2aGe20OuUAU', 'https://www.youtube.com/watch?vJJXEhmcQxwg', 'https://www.youtube.com/watch?vvg8yizC4W8E']
['2025-05-03 11:56:05', '2025-05-01 23:43:43', '2025-05-05 04:19:47', '2025-04-25 04:07:48', '2025-05-02 08:31:03']


In [42]:
# Create a dictionary to store channel information for each video ID
channel_info_dict = {}

# Fetch channel information for each video ID
for video_id in ids:
    info = get_channel_info(video_id)
    if info:
        channel_info_dict[video_id] = info

# Create a DataFrame using the collected data
df = pd.DataFrame([
    {
        'Video ID': video_id,
        'Video URL': urls[idx],
        'Video Creation Date': created_dates[idx],
        'Video Title': info['title'],
        'Channel ID': info['channelId'],
        'Channel Title': info['channelTitle']
    } for idx, (video_id, info) in enumerate(channel_info_dict.items())
])

# Define the output file path
p = Path("~/Downloads") / "Watch later videos channel info.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(p.expanduser(), index=False, engine='openpyxl')
print(f"Channel info saved to {p.expanduser()}")

Channel info saved to /Users/hochiyau/Downloads/Watch later videos channel info.xlsx


In [43]:
df.to_excel(p.expanduser(), index=False, engine='openpyxl')
print(f"Channel info saved to {p.expanduser()}")

Channel info saved to /Users/hochiyau/Downloads/Watch later videos channel info.xlsx
